In [ ]:
import pandas as pd
import numpy as np
from google.cloud import bigquery
import pickle
from datetime import date, datetime, timezone


import warnings
warnings.filterwarnings("ignore")

In [ ]:
def run_query(query): 
    # Set up the BigQuery client
    project_id = 'som-nero-phi-sywang-starr'
    client = bigquery.Client(project=project_id)

    # Execute the query
    df = client.query(query, project=project_id).to_dataframe()

    return df

## Load Initial Cohort

In [ ]:
cohort = pd.read_csv("processed_data/cohort.csv", dtype={'MRN': 'string'})

In [ ]:
cohort.head()

In [ ]:
print(f'There is a total of {len(cohort)} patients in this cohort.')

## Load Measurement Data

In [ ]:
query = """
SELECT 
    co.measurement_id,
    co.person_id,
    co.measurement_concept_id,
    co.measurement_datetime,
    co.value_as_number,
    mc.*
FROM `som-nero-phi-sywang-starr.gps_stanford_clinic.measurement` AS co
LEFT JOIN `som-nero-phi-sywang-starr.gps_stanford_clinic.mrn_crosswalk` AS mc
ON co.person_id = mc.person_id;

"""

lab_dat = run_query(query)
lab_dat = lab_dat.drop(columns = ['person_id','person_id_1', 'source_dob'], axis = 1, inplace = False)
print(f"# of rows: {len(lab_dat)}")
print(f"# of unique pats: {len(lab_dat['MRN'].unique())}")

In [ ]:
lab_dat.head()

In [ ]:
x = lab_dat.merge(cohort, on='MRN', how='left')
filtered_x = x[(x.measurement_datetime <= x.diag_date) | x.diag_date.isna()]
filtered_x.drop(columns = ['measurement_id', 'outcome', 'diag_date'], axis = 1, inplace = True)

In [ ]:
filtered_x.head()

In [ ]:
lab_data = filtered_x

## Extract Features

### Helper Functions

In [ ]:
def get_unique_data(concept_id, col_name, keep_date = False, data_col_name = None):
    df = lab_data[lab_data.measurement_concept_id == concept_id][['MRN', 'value_as_number',
                                                                  'measurement_datetime']]
    df.rename(columns={'value_as_number': col_name}, inplace=True)
    if keep_date:
        df.rename(columns={'measurement_datetime': data_col_name}, inplace=True)
        
    df = df[df[col_name].notna()]

    unique_pats_before = len(df['MRN'].unique())
    
    if keep_date:
        df = df.sort_values(data_col_name).groupby('MRN').last().reset_index()
    else:
        df = df.sort_values('measurement_datetime').groupby('MRN').last().reset_index()
    
    unique_pats_after = len(df['MRN'].unique())
    assert unique_pats_before == unique_pats_after
    
    if not keep_date:
        df.drop(['measurement_datetime'], axis = 1, inplace = True)
    print(f'Unique Pats: {unique_pats_after}')
    return df

In [ ]:
def combine_cols(df1, df2, df1_col, df2_col, df1_date_col, df2_date_col, col_name,
                 keep_date = False, date_col = None):
    df1[df1_date_col] = pd.to_datetime(df1[df1_date_col])

    if df1[df1_date_col].dt.tz is None:  # If naive, localize
        df1[df1_date_col] = df1[df1_date_col].dt.tz_localize('UTC')
    else:  # If already aware, convert to UTC
        df1[df1_date_col] = df1[df1_date_col].dt.tz_convert('UTC')

    df2[df2_date_col] = pd.to_datetime(df2[df2_date_col])

    if df2[df2_date_col].dt.tz is None:
        df2[df2_date_col] = df2[df2_date_col].dt.tz_localize('UTC')
    else:
        df2[df2_date_col] = df2[df2_date_col].dt.tz_convert('UTC')

    
    combined_df = pd.merge(df1, df2, how='outer', on = 'MRN')
    dt = (datetime.now(timezone.utc))
    
    combined_df[df1_date_col] = combined_df[df1_date_col].fillna(dt.replace(year=dt.year-200))
    combined_df[df2_date_col] = combined_df[df2_date_col].fillna(dt.replace(year=dt.year-200))
    combined_df[col_name] = np.where(combined_df[df2_date_col] > combined_df[df1_date_col],
                              combined_df[df2_col], combined_df[df1_col])
    
    if keep_date:
        combined_df[date_col] = np.where(combined_df[df2_date_col] > combined_df[df1_date_col],
                          combined_df[df2_date_col], combined_df[df1_date_col])
    
    combined_df.drop([df2_date_col, df1_date_col, df2_col, df1_col],axis = 1, inplace = True)
    
    unique_pats = len(combined_df['MRN'].unique())
    print(f'Unique Pats: {unique_pats}')
    print(combined_df.head())
    
    return combined_df

### BMI, BP, Heart Rate

In [ ]:
# bmi
bmi = get_unique_data(3038553, 'bmi', keep_date = False, data_col_name = None)
bmi.head()

In [ ]:
# diastolic bp
dbp = get_unique_data(3012888, 'dbp', keep_date = False, data_col_name = None)
dbp.head()

In [ ]:
# systolic bp
sbp = get_unique_data(3004249, 'sbp', keep_date = False, data_col_name = None)
sbp.head()

In [ ]:
# heart rate
heart_rate = get_unique_data(3027018, 'heart_rate', keep_date = False, data_col_name = None)
heart_rate.head()

In [ ]:
lab_df = pd.merge(sbp, dbp, how='outer', on = 'MRN')
lab_df = pd.merge(lab_df, heart_rate, how='outer', on = 'MRN')
lab_df = pd.merge(lab_df, bmi, how='outer', on = 'MRN')

unique_pats = len(lab_df['MRN'].unique())
print(f'Unique Pats: {unique_pats}')
lab_df.head()

### Hemoglobin A1C

In [ ]:
a1c = get_unique_data(3004410, 'a1c', keep_date = False, data_col_name = None)
a1c.head()

In [ ]:
lab_df = pd.merge(lab_df, a1c, how='outer', on = 'MRN')
unique_pats = len(lab_df['MRN'].unique())
print(f'Unique Pats: {unique_pats}')
lab_df.head()

### Thyroid-Stimulating Hormone (TSH)

In [ ]:
tsh = get_unique_data(3009201, 'tsh', keep_date = False, data_col_name = None)
tsh.head()

In [ ]:
lab_df = pd.merge(lab_df, tsh, how='outer', on = 'MRN')
unique_pats = len(lab_df['MRN'].unique())
print(f'Unique Pats: {unique_pats}')
lab_df.head()

### Lipid Panel

In [ ]:
# Total Cholestrol
total_chol = get_unique_data(3027114, 'total_chol', keep_date = False, data_col_name = None)
total_chol.head()

In [ ]:
# (LDL) Cholesterol
ldl_chol = get_unique_data(3028288, 'ldl_chol', keep_date = False, data_col_name = None)
ldl_chol.head()

In [ ]:
# (HDL) Cholesterol
hdl_chol = get_unique_data(3007070, 'hdl_chol', keep_date = False, data_col_name = None)
hdl_chol.head()

In [ ]:
# (Non-HDL) Cholesterol
nonhdl_chol = get_unique_data(3044491, 'nonhdl_chol', keep_date = False, data_col_name = None)
nonhdl_chol.head()

In [ ]:
# Triglyceride
triglyceride = get_unique_data(3022192, 'triglyceride', keep_date = False, data_col_name = None)
triglyceride.head()

In [ ]:
lipid = pd.merge(total_chol, ldl_chol, how='outer', on = 'MRN')
lipid = pd.merge(lipid, hdl_chol, how='outer', on = 'MRN')
lipid = pd.merge(lipid, nonhdl_chol, how='outer', on = 'MRN')
lipid = pd.merge(lipid, triglyceride, how='outer', on = 'MRN')
print(f'Unique Pats: {unique_pats}')
lipid.head()

In [ ]:
lab_df = pd.merge(lab_df, lipid, how='outer', on = 'MRN')
unique_pats = len(lab_df['MRN'].unique())
print(f'Unique Pats: {unique_pats}')
lab_df.head()

### CBC Panel

In [ ]:
# Red Blood Cells
red_blood = get_unique_data(3020416, 'red_blood', keep_date = False, data_col_name = None)
red_blood.head()

In [ ]:
# White Blood Cells
white_blood = get_unique_data(3000905, 'white_blood', keep_date = False, data_col_name = None)
white_blood.head()

In [ ]:
# Hemoglobin
hemoglobin = get_unique_data(3000963, 'hemoglobin', keep_date = False, data_col_name = None)
hemoglobin.head()

In [ ]:
# Hematocrit
hematocrit = get_unique_data(3023314, 'hematocrit', keep_date = False, data_col_name = None)
hematocrit.head()

In [ ]:
# Platelets
platelets = get_unique_data(3024929, 'platelets', keep_date = False, data_col_name = None)
platelets.head()

In [ ]:
cbc = pd.merge(white_blood, red_blood, how='outer', on = 'MRN')
cbc = pd.merge(cbc, hemoglobin, how='outer', on = 'MRN')
cbc = pd.merge(cbc, hematocrit, how='outer', on = 'MRN')
cbc = pd.merge(cbc, platelets, how='outer', on = 'MRN')
unique_pats = len(cbc['MRN'].unique())
print(f'Unique Pats: {unique_pats}')
cbc.head()

In [ ]:
lab_df = pd.merge(lab_df, cbc, how='outer', on = 'MRN')
unique_pats = len(lab_df['MRN'].unique())
print(f'Unique Pats: {unique_pats}')
lab_df.head()

### Comprehensive Metabolic Panel (CMP)

In [ ]:
# Sodium
sodium_1 = get_unique_data(3000285, 'sodium_1', keep_date = True, data_col_name = 'measurement_datetime_1')
sodium_2 = get_unique_data(3019550, 'sodium_2', keep_date = True, data_col_name = 'measurement_datetime_2')

In [ ]:
sodium = combine_cols(sodium_1, sodium_2, 'sodium_1', 'sodium_2', 
             'measurement_datetime_1', 'measurement_datetime_2', 'sodium', keep_date = False)

In [ ]:
# Potassium
potassium_1 = get_unique_data(3005456, 'potassium_1', keep_date = True, data_col_name = 'measurement_datetime_1')
potassium_2 = get_unique_data(3023103, 'potassium_2', keep_date = True, data_col_name = 'measurement_datetime_2')

In [ ]:
potassium = combine_cols(potassium_1, potassium_2, 'potassium_1', 'potassium_2', 
             'measurement_datetime_1', 'measurement_datetime_2', 'potassium', keep_date = False)

In [ ]:
# Chloride
chloride_1 = get_unique_data(3018572, 'chloride_1', keep_date = True, data_col_name = 'measurement_datetime_1')
chloride_2 = get_unique_data(3014576, 'chloride_2', keep_date = True, data_col_name = 'measurement_datetime_2')

In [ ]:
chloride = combine_cols(chloride_1, chloride_2, 'chloride_1', 'chloride_2', 
             'measurement_datetime_1', 'measurement_datetime_2', 'chloride', keep_date = False)

In [ ]:
# Cardon Dioxide
co2_1 = get_unique_data(3014094, 'co2_1', keep_date = True, data_col_name = 'measurement_datetime_1')
co2_2 = get_unique_data(3015632, 'co2_2', keep_date = True, data_col_name = 'measurement_datetime_2')

In [ ]:
co2 = combine_cols(co2_1, co2_2, 'co2_1', 'co2_2', 
             'measurement_datetime_1', 'measurement_datetime_2', 'co2', keep_date = False)

In [ ]:
# Albumin
albumin = get_unique_data(3024561, 'albumin', keep_date = False)
albumin.head()

In [ ]:
# Alkaline Phosphatase
alk_phos_1 = get_unique_data(3001110, 'alk_phos_1', keep_date = True, data_col_name = 'measurement_datetime_1')
alk_phos_2 = get_unique_data(3035995, 'alk_phos_2', keep_date = True, data_col_name = 'measurement_datetime_2')

In [ ]:
alk_phos = combine_cols(alk_phos_1, alk_phos_2, 'alk_phos_1', 'alk_phos_2', 
             'measurement_datetime_1', 'measurement_datetime_2', 'alk_phos', keep_date = False)

In [ ]:
# Bilirubin
bilirubin_1 = get_unique_data(3028833, 'bilirubin_1', keep_date = True, data_col_name = 'measurement_datetime_1')
bilirubin_2 = get_unique_data(3024128, 'bilirubin_2', keep_date = True, data_col_name = 'measurement_datetime_2')

In [ ]:
bilirubin = combine_cols(bilirubin_1, bilirubin_2, 'bilirubin_1', 'bilirubin_2', 
             'measurement_datetime_1', 'measurement_datetime_2', 'bilirubin', keep_date = False)

In [ ]:
# Aspartate Transaminase
aspartate_trans_1a = get_unique_data(3013721, 'aspartate_trans_1a', keep_date = True,
                                     data_col_name = 'measurement_datetime_1a')
aspartate_trans_1b = get_unique_data(36305398, 'aspartate_trans_1b', keep_date = True,
                                     data_col_name = 'measurement_datetime_2a')
aspartate_trans_1 = combine_cols(aspartate_trans_1a, aspartate_trans_1b, 'aspartate_trans_1a', 'aspartate_trans_1b', 
             'measurement_datetime_1a', 'measurement_datetime_2a', 'aspartate_trans_1', keep_date = True,
                                 date_col = 'measurement_datetime_1')

In [ ]:
aspartate_trans_2 = get_unique_data(3037081, 'aspartate_trans_2',
                                    keep_date = True, data_col_name = 'measurement_datetime_2')
aspartate_trans = combine_cols(aspartate_trans_1, aspartate_trans_2, 'aspartate_trans_1', 'aspartate_trans_2', 
             'measurement_datetime_1', 'measurement_datetime_2', 'aspartate_trans', keep_date = False)

In [ ]:
# Alanine Transaminase
alaine_trans_1a = get_unique_data(46235106, 'alaine_trans_1a', keep_date = True,
                                     data_col_name = 'measurement_datetime_1a')
alaine_trans_1b = get_unique_data(3006923, 'alaine_trans_1b', keep_date = True,
                                     data_col_name = 'measurement_datetime_1b')

alaine_trans_1 = combine_cols(alaine_trans_1a, alaine_trans_1b, 'alaine_trans_1a', 'alaine_trans_1b', 
             'measurement_datetime_1a', 'measurement_datetime_1b', 'alaine_trans_1', keep_date = True,
                                 date_col = 'measurement_datetime_1')

In [ ]:
alaine_trans_2a = get_unique_data(3027388, 'alaine_trans_2a', keep_date = True,
                                     data_col_name = 'measurement_datetime_2a')
alaine_trans_2b = get_unique_data(3005755, 'alaine_trans_2b', keep_date = True,
                                     data_col_name = 'measurement_datetime_2b')

alaine_trans_2 = combine_cols(alaine_trans_2a, alaine_trans_2b, 'alaine_trans_2a', 'alaine_trans_2b', 
             'measurement_datetime_2a', 'measurement_datetime_2b', 'alaine_trans_2', keep_date = True,
                                 date_col = 'measurement_datetime_2')

In [ ]:
alaine_trans = combine_cols(alaine_trans_1, alaine_trans_2, 'alaine_trans_1', 'alaine_trans_2', 
             'measurement_datetime_1', 'measurement_datetime_2', 'alaine_trans', keep_date = False)

In [ ]:
# Blood Urea Nitrogen
blood_urea_nit = get_unique_data(3013682, 'blood_urea_nit', keep_date = False)
blood_urea_nit.head()

In [ ]:
# Total Protein
protein = get_unique_data(3020630, 'protein', keep_date = False)
protein.head()

In [ ]:
# Calcium
calcium = get_unique_data(3006906, 'calcium', keep_date = False)
calcium.head()

In [ ]:
# Creatinine
creatinine = get_unique_data(3016723, 'creatinine', keep_date = False)
creatinine.head()

In [ ]:
# Glucose
glucose_1 = get_unique_data(3000483, 'glucose_1', keep_date = True, data_col_name = 'measurement_datetime_1')
glucose_2 = get_unique_data(3004501, 'glucose_2', keep_date = True, data_col_name = 'measurement_datetime_2')

In [ ]:
glucose = combine_cols(glucose_1, glucose_2, 'glucose_1', 'glucose_2', 
             'measurement_datetime_1', 'measurement_datetime_2', 'glucose', keep_date = False)

In [ ]:
cmp = pd.merge(sodium, potassium, how='outer', on = 'MRN')
cmp = pd.merge(cmp, chloride, how='outer', on = 'MRN')
cmp = pd.merge(cmp, co2, how='outer', on = 'MRN')
cmp = pd.merge(cmp, albumin, how='outer', on = 'MRN')
cmp = pd.merge(cmp, alk_phos, how='outer', on = 'MRN')

cmp = pd.merge(cmp, bilirubin, how='outer', on = 'MRN')
cmp = pd.merge(cmp, aspartate_trans, how='outer', on = 'MRN')
cmp = pd.merge(cmp, alaine_trans, how='outer', on = 'MRN')

cmp = pd.merge(cmp, blood_urea_nit, how='outer', on = 'MRN')
cmp = pd.merge(cmp, protein, how='outer', on = 'MRN')
cmp = pd.merge(cmp, calcium, how='outer', on = 'MRN')
cmp = pd.merge(cmp, creatinine, how='outer', on = 'MRN')
cmp = pd.merge(cmp, glucose, how='outer', on = 'MRN')

unique_pats = len(cmp['MRN'].unique())
print(f'Unique Pats: {unique_pats}')
cmp.head()

In [ ]:
lab_df = pd.merge(lab_df, cmp, how='outer', on = 'MRN')
unique_pats = len(lab_df['MRN'].unique())
print(f'Unique Pats: {unique_pats}')
lab_df.head()

### Urinalysis

In [ ]:
# PH
ph_1a = get_unique_data(3015501, 'ph_1a', keep_date = True, data_col_name = 'measurement_datetime_1a')
ph_1b = get_unique_data(3015736, 'ph_1b', keep_date = True, data_col_name = 'measurement_datetime_1b')
ph_1 = combine_cols(ph_1a, ph_1b, 'ph_1a', 'ph_1b', 
             'measurement_datetime_1a', 'measurement_datetime_1b', 'ph_1',
             keep_date = True, date_col = 'ph_1_date')

In [ ]:
ph_2a = get_unique_data(3029305, 'ph_2a', keep_date = True, data_col_name = 'measurement_datetime_2a')
ph_2b = get_unique_data(3022621, 'ph_2b', keep_date = True, data_col_name = 'measurement_datetime_2b')
ph_2 = combine_cols(ph_2a, ph_2b, 'ph_2a', 'ph_2b', 
             'measurement_datetime_2a', 'measurement_datetime_2b', 'ph_2',
             keep_date = True, date_col = 'ph_2_date')

In [ ]:
ph = combine_cols(ph_1, ph_2, 'ph_1', 'ph_2','ph_1_date', 'ph_2_date', 'ph_urine')

In [ ]:
lab_df = pd.merge(lab_df, ph, how='outer', on = 'MRN')
unique_pats = len(lab_df['MRN'].unique())
print(f'Unique Pats: {unique_pats}')
lab_df.head()

## Save File

In [ ]:
lab_df.to_csv('processed_data/lab_raw.csv', index=False)